# Biostat 257 Homework 3

**Due May 6 @ 11:59PM**

In [ ]:
versioninfo()

In [1]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

**Solution**

$$
l(\beta, \Sigma, \sigma^2) = - \frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log|Z_i \Sigma Z_i' + \sigma^2 I_{n_i}| - \frac{1}{2} (Y_i - X_i \beta)' (Z_i \Sigma Z_i' + \sigma^2 I_{n_i})^{-1} (Y_i - X_i \beta)
$$

Let $\Sigma = L L'$, and $Q = I_q + \frac{1}{\sigma^2} L'Z_i'Z_i L$.

\begin{align}
|Z_i \Sigma Z_i' + \sigma^2 I_{n_i}| &= |Z_i L L' Z_i' + \sigma^2 I_{n_i}| \\
& = (\sigma^2)^{n_i} |Q| \\
(Z_i \Sigma Z_i' + \sigma^2 I_{n_i})^{-1} &= (Z_i L L' Z_i' + \sigma^2 I_{n_i})^{-1} \\
&= \frac{1}{\sigma^2} [I_{n_i} - \frac{1}{\sigma^2} Z_i L Q^{-1} L' Z_i')]
\end{align}

\begin{align*}
l(\beta, \Sigma, \sigma^2) = &- \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2} \log |Q| \\ 
& - \frac{1}{2 \sigma^2} (Y_i - X_i \beta)' [I_{n_i} - \frac{1}{\sigma^2} Z_i L Q^{-1} L' Z_i')] (Y_i - X_i \beta) \\
= & - \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2} \log |Q| \\
&- \frac{1}{2 \sigma^2} [(Y_i - X_i \beta)' (Y_i - X_i \beta) - \frac{1}{\sigma^2} (Y_i - X_i \beta)' Z_i L Q^{-1} L' Z_i' (Y_i - X_i \beta) ] \\
= & - \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2} \log |Q| - \frac{1}{2 \sigma^2} \{(Y_i - X_i \beta)' (Y_i - X_i \beta) \\
& - \frac{1}{\sigma^2} [Y_i' Z_i L Q^{-1} L' Z_i' Y_i - 2 Y_i' Z_i L Q^{-1} L' Z_i' X_i \beta + \beta' X_i' Z_i L Q^{-1} L' Z_i' X_i \beta] \}
\end{align*}

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [2]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p   :: Vector{T}
    storage_q   :: Vector{T}
    xtx         :: Matrix{T}
    ztx         :: Matrix{T}
    ztz         :: Matrix{T}
    storage_qq  :: Matrix{T}
    # add new ones
    zty         :: Vector{T}
    storage_LQL :: Matrix{T}
    r           :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p   = Vector{T}(undef, size(X, 2))
    storage_q   = Vector{T}(undef, size(Z, 2))
    xtx         = transpose(X) * X
    ztx         = transpose(Z) * X
    ztz         = transpose(Z) * Z
    storage_qq  = similar(ztz)
    # add new ones
    zty         = transpose(Z) * y
    storage_LQL = similar(ztz)
    r           = similar(y)
    
    LmmObs(y, X, Z, storage_p, 
           storage_q, 
           xtx, ztx, ztz, storage_qq, zty, storage_LQL, r)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

**Solution**

In [5]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood
    
    obs.storage_qq .= I + 1 / σ² * L' * obs.ztz * L
    obs.r .= y - X * β
    
    # obs.storage_q .= L' * Z' * r
    # obs.storage_q = Z' * (Y - X * β)
    # gemv!(tA, alpha, A, x, beta, y)
    BLAS.gemv!('T', 1.0, Z, obs.r, 0.0, obs.storage_q)
    # obs.storage_q = L' * obs.storage_q
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)
    
    
    logL = - n//2 * log(2π) - n//2 * log(σ²) - 1//2 * logdet(obs.storage_qq) - 
            # 1 // 2 / σ² * (y - X * β)' * (y - X * β) + 
            1 // 2 / σ² * norm(obs.r)^2 + 
            1 // 2 / σ²^2 * obs.storage_q' * inv(obs.storage_qq) * obs.storage_q
    
    return logL
end

L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058294

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [4]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [0.0, 0.0, 0.0, 0.0, 1.0e-323], [2.2495529796e-314, 2.2156766294e-314, 2.2831621153e-314], [2000.0 -16.870943820386746 … -4.6787564876784

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [10]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [11]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058303

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [12]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [13]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 7634 samples with 1 evaluation.
 Range (min … max):  589.270 μs …   8.738 ms  ┊ GC (min … max): 0.00% … 91.24%
 Time  (median):     636.624 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   649.059 μs ± 151.880 μs  ┊ GC (mean ± σ):  0.42% ±  1.79%

         ▅▆▇▇█▆▅▄▃▁                                              
  ▂▂▂▂▂▃▆██████████▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂ ▃
  589 μs           Histogram: frequency by time          820 μs <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [14]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.441 μs … 192.082 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.870 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.540 μs ±   5.082 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▄▂▁▁▄▃▂▁▂▂▂▁▁▁▁▁                                          ▁
  ████████████████████████████▇█▇█▇▇▇▇▇▆▆▇▇▇▇▆▆▆▅▅▅▅▄▄▅▅▅▄▄▅▄ █
  6.44 μs      Histogram: log(frequency) by time      24.3 μs <

 Memory estimate: 544 bytes, allocs estimate: 8.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [15]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

2.780177157330849

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [ ]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.